In [1]:
import os
import sys
import shutil
import numpy as np
sys.path.insert(0, '../scripts')

from eeg_cnn_lib import transfer_all_class, prepare_data_labels
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import optimizers

# The images are in a folder named 'shapes/training'
train_dir_name = '../../../data/classification/valence/train'
validation_dir_name = '../../../data/classification/valence/validation'
test_dir_name = '../../../data/classification/valence/test'


# All images are 32 pixels
img_size = (32,32)
img_size_rgba = (32,32,4)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(train_dir_name))

Using TensorFlow backend.


In [2]:
import sys
import keras
print('Keras version:',keras.__version__)

from keras import backend as K

Keras version: 2.3.1


In [3]:
transfer_all_class(test_dir_name,train_dir_name,1.0,classes)
transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
transfer_all_class(train_dir_name,test_dir_name,0.2,classes)

In [4]:
X, Y = prepare_data_labels(classes, train_dir_name)
X=np.asarray(X)
Y=np.asarray(Y)

In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(128, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))

model.add(Dense(len(classes), activation='softmax'))

opt = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        1184      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)             

In [6]:
kf = KFold(n_splits=10, shuffle=False)
accuracies = []

In [ ]:
for train_index, val_index in kf.split(X, Y):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
    for index in range(len(X_val)):
        class_label=''
        if(Y_val[index]==0):
            class_label=classes[0]
        else:
            class_label=classes[1]  
        #Then, copy the validation images to the validation folder
        shutil.move(train_dir_name+"/"+class_label+'/'+X_val[index], 
                    validation_dir_name+"/"+class_label+'/'+X_val[index])
        
    batch_size = 256
    epoch=100
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_directory(
        train_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode=None)
        
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
        
    predictions = model.predict_generator(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    y_true = validation_generator.classes
    accuracy=accuracy_score(y_true, y_predictions)
    accuracies.append(accuracy)
    print(accuracy)

Found 12241 images belonging to 2 classes.
Found 1361 images belonging to 2 classes.
Epoch 1/100
48/48 [==============================] - 8s 174ms/step - loss: 0.6989 - accuracy: 0.5627
Epoch 2/100
48/48 [==============================] - 8s 166ms/step - loss: 0.6812 - accuracy: 0.5814
Epoch 3/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6801 - accuracy: 0.5814
Epoch 4/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6796 - accuracy: 0.5814
Epoch 5/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6794 - accuracy: 0.5814
Epoch 6/100
48/48 [==============================] - 8s 161ms/step - loss: 0.6789 - accuracy: 0.5814
Epoch 7/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6772 - accuracy: 0.5809
Epoch 8/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6763 - accuracy: 0.5819
Epoch 9/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6758 - accuracy: 0.5815
Epoch 

48/48 [==============================] - 8s 164ms/step - loss: 0.2477 - accuracy: 0.8793
Epoch 81/100
48/48 [==============================] - 8s 159ms/step - loss: 0.2498 - accuracy: 0.8838
Epoch 82/100
48/48 [==============================] - 8s 159ms/step - loss: 0.2443 - accuracy: 0.8849
Epoch 83/100
48/48 [==============================] - 8s 158ms/step - loss: 0.2411 - accuracy: 0.8868
Epoch 84/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2336 - accuracy: 0.8957
Epoch 85/100
48/48 [==============================] - 8s 159ms/step - loss: 0.2319 - accuracy: 0.8913
Epoch 86/100
48/48 [==============================] - 28s 574ms/step - loss: 0.2295 - accuracy: 0.8936
Epoch 87/100
48/48 [==============================] - 13s 264ms/step - loss: 0.2335 - accuracy: 0.8904
Epoch 88/100
48/48 [==============================] - 10s 211ms/step - loss: 0.2182 - accuracy: 0.8984
Epoch 89/100
48/48 [==============================] - 9s 188ms/step - loss: 0.2225 - accurac

48/48 [==============================] - 8s 159ms/step - loss: 0.1297 - accuracy: 0.9431
Epoch 59/100
48/48 [==============================] - 8s 158ms/step - loss: 0.1302 - accuracy: 0.9423
Epoch 60/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1335 - accuracy: 0.9403
Epoch 61/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1243 - accuracy: 0.9435
Epoch 62/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1336 - accuracy: 0.9413
Epoch 63/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1273 - accuracy: 0.9433
Epoch 64/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1354 - accuracy: 0.9413
Epoch 65/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1282 - accuracy: 0.9431
Epoch 66/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1227 - accuracy: 0.9485
Epoch 67/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1127 - accuracy: 

48/48 [==============================] - 8s 172ms/step - loss: 0.0948 - accuracy: 0.9610
Epoch 37/100
48/48 [==============================] - 8s 165ms/step - loss: 0.1043 - accuracy: 0.9570
Epoch 38/100
48/48 [==============================] - 8s 160ms/step - loss: 0.1060 - accuracy: 0.9559
Epoch 39/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1018 - accuracy: 0.9561
Epoch 40/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1019 - accuracy: 0.9584
Epoch 41/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0971 - accuracy: 0.9585
Epoch 42/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0961 - accuracy: 0.9587
Epoch 43/100
48/48 [==============================] - 8s 158ms/step - loss: 0.1078 - accuracy: 0.9553
Epoch 44/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0948 - accuracy: 0.9601
Epoch 45/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0974 - accuracy: 

Epoch 15/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0924 - accuracy: 0.9621
Epoch 16/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0855 - accuracy: 0.9626
Epoch 17/100
48/48 [==============================] - 8s 166ms/step - loss: 0.0933 - accuracy: 0.9612
Epoch 18/100
48/48 [==============================] - 8s 164ms/step - loss: 0.0951 - accuracy: 0.9610
Epoch 19/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0859 - accuracy: 0.9645
Epoch 20/100
48/48 [==============================] - 8s 163ms/step - loss: 0.0868 - accuracy: 0.9630
Epoch 21/100
48/48 [==============================] - 8s 164ms/step - loss: 0.0831 - accuracy: 0.9656
Epoch 22/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0849 - accuracy: 0.9635
Epoch 23/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0837 - accuracy: 0.9634
Epoch 24/100
48/48 [==============================] - 8s 166ms/step - loss: 0.0806

...

Epoch 31/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0463 - accuracy: 0.9818
Epoch 32/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0510 - accuracy: 0.9817
Epoch 33/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0488 - accuracy: 0.9798
Epoch 34/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0517 - accuracy: 0.9797
Epoch 35/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0493 - accuracy: 0.9807
Epoch 36/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0485 - accuracy: 0.9825
Epoch 37/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0510 - accuracy: 0.9813
Epoch 38/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0482 - accuracy: 0.9810
Epoch 39/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0527 - accuracy: 0.9790
Epoch 40/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0540

48/48 [==============================] - 8s 159ms/step - loss: 0.0497 - accuracy: 0.9821
Epoch 10/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0600 - accuracy: 0.9770
Epoch 11/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0508 - accuracy: 0.9817
Epoch 12/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0458 - accuracy: 0.9816
Epoch 13/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0418 - accuracy: 0.9833
Epoch 14/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0439 - accuracy: 0.9842
Epoch 15/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0412 - accuracy: 0.9846
Epoch 16/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0404 - accuracy: 0.9868
Epoch 17/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0463 - accuracy: 0.9829
Epoch 18/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0439 - accuracy: 

48/48 [==============================] - 8s 158ms/step - loss: 0.0352 - accuracy: 0.9862
Epoch 90/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0428 - accuracy: 0.9825
Epoch 91/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0406 - accuracy: 0.9859
Epoch 92/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0411 - accuracy: 0.9845
Epoch 93/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0417 - accuracy: 0.9847
Epoch 94/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0506 - accuracy: 0.9821
Epoch 95/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0424 - accuracy: 0.9843
Epoch 96/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0435 - accuracy: 0.9856
Epoch 97/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0421 - accuracy: 0.9849
Epoch 98/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0373 - accuracy: 

Epoch 68/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0410 - accuracy: 0.9845
Epoch 69/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0434 - accuracy: 0.9848
Epoch 70/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0421 - accuracy: 0.9842
Epoch 71/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0455 - accuracy: 0.9821
Epoch 72/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0370 - accuracy: 0.9869
Epoch 73/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0499 - accuracy: 0.9826
Epoch 74/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0469 - accuracy: 0.9833
Epoch 75/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0409 - accuracy: 0.9856
Epoch 76/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0415 - accuracy: 0.9848
Epoch 77/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0393

48/48 [==============================] - 8s 159ms/step - loss: 0.0417 - accuracy: 0.9851
Epoch 47/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0354 - accuracy: 0.9879
Epoch 48/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0343 - accuracy: 0.9870
Epoch 49/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0391 - accuracy: 0.9858
Epoch 50/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0375 - accuracy: 0.9866
Epoch 51/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0431 - accuracy: 0.9849
Epoch 52/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0401 - accuracy: 0.9873
Epoch 53/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0338 - accuracy: 0.9872
Epoch 54/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0379 - accuracy: 0.9859
Epoch 55/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0383 - accuracy: 

In [ ]:
np.average(accuracies)

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir_name,
    target_size=img_size,
    color_mode='rgba',
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
loss, acc = model.evaluate_generator(test_generator)
acc